In [ ]:
import simpy
import pandas as pd
import numpy as np 
from collections import namedtuple
#from recordtype import recordtype
import random 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from prettytable import PrettyTable
#get_ipython().magic('matplotlib inline')
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 
from collections import deque 
from itertools import product
import math

class DQN:#모델 선언
    def __init__(self): #parameter들의 초기값
        #self.gamma = 0.85
        self.epsilon = 0.999
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.95
        self.tau = 0.125 #?
        self.learning_rate = 1
        self.memory = deque(maxlen= 2000)
        self.model = self.create_model()
        self.target_model = self.create_model() #target_model이란 create_model의 return값..?



    # create the neural network to train the q function 
    def create_model(self): #Q값예측모델. input:state 1개 output:action 10개?
        model = Sequential()
        model.add(Dense(24, input_dim= 3, activation= 'relu')) # input은 state
        model.add(Dense(48, activation= 'relu'))
        model.add(Dense(24, activation= 'relu'))
        model.add(Dense(66)) #계산했을 때,(1~10)까지 세 수의 합이 10이 되는 경우의수는 63개. output에 대한 가중치는 매번 update되기 때문에 이에 mapping시키면 된다. 
        model.compile(loss= 'mean_squared_error', optimizer= Adam(lr= self.learning_rate))
        
        return model 



    # Action function to choose the best action given the q-function if not exploring based on epsilon p값에 의한 예측이 아닐때
    def choose_action(self, state, allowed_actions): #action을 선택 (parameter로 선택가능한 action이 들어옴)
        #각 source별 allowed action이 매우 많은데 output은 source별 q값인지? 아니면 각 가능한 action별 q값인지?
        
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        r = np.random.random()
        if r < self.epsilon: #p값보다 작은 경우 랜덤한 액션을 취함
            print("random action")
            return random.choice(allowed_actions)
#             for i in range(len(allowed_actions)):
#                 random.choice(allowed_actions)
                
#                 if ((rnd[0]>=state[0])and(rnd[1]>=state[1])and(rnd[2]>=state[2])):
#                     break                         
                                        
#             return rnd #가능한 action중 랜덤하게 선택
        
        state = np.array(state).reshape(1,len(state)) #p값보다 큰경우, state 배열 생성

        pred = self.model.predict(state)[0]
        
        
            
        #state=0이면 filesize=0이라는 뜻인데 이 때 action을 0으로 할당해줘야 함. 만약 1개의 flow가 완료된경우는 어떻게 분배할 지 애매 
        if (0 in state[0]):
            print("state",state[0])
             #전송완료된 flow가 하나라도 있다면(근데 state가 모두 000인경우...)
            index = [i for i, value in enumerate(state[0]) if value == 0]
            
            try:
                if (index[2]==2):
                    return [3,3,4] #state가 다 0이면
                
            except:
                pass 
            
            q=[]
            # indexes = [i for i, value in enumerate(a) if value[s_] = 0 for s_ in s]
            # indexes = [i for s_ in s if value[s_]==0 ]
            
            for a in allowed_actions:
                try:
                    if (a[index[0]]==0): #0인 state가 하나면 여기까지만        
                        try:
                            if(a[index[1]]==0):
                                try:
                                    if(a[index[2]]==0): #index가 2까지있다는것은 모든 인덱스가 0이라는것
                                        #아무것도 없음
                                        pass
                                        
                                except: 
                                    q.append(allowed_actions.index(a))
                                    continue
                        except:
                            q.append(allowed_actions.index(a))
                            continue
                except:continue
            print ("q:",q)
            preds=[pred[q_] for q_ in q]
            allowedacts=[allowed_actions[q_] for q_ in q ] #그 actions중에 가장 큰 Q값을 가지는 action을 가져옴
            return allowedacts[np.argmax(preds)]
        
        return self.maxQ_action(pred,state,allowed_actions) #Q예측값중 min_rate 이상으로 가장 큰 action을 선택
    

    def maxQ_action(self,pred,state,allowed_actions):#allowed action 생성 (min_rate 이상 조합만 남김)
   
        #for p in range(len(pred)):
#             index=np.argmax(pred)
#             if ((allowed_actions[index][0]>=state[0][0])and(allowed_actions[index][1]>=state[0][1])and(allowed_actions[index][2]>=state[0][2])):#모든 action이 min_rate보다 크거나 같으면
#                 #print("선택된 action set은:",allowed_actions[index])
#                 return allowed_actions[index]
#             else:
#                 pred[index]=np.min(pred)
                
        #만약 return되지 못한 경우(즉, pred값이 가장 큰 index가 min_rate 조건을 충족시키지 못할 경우) 
        #위 코드를 주석처리하면 min_rate에 신경쓰지 않고 action을 할당하게 되는 코드 -> allowed_action이 none으로 return되는 상황 발생
        return allowed_actions[np.argmax(pred)]
        
        
        
    # create replay buffer memory to sample randomly #메모리에서 꺼내서 학습할 수 있게 저장
    def remember(self, state, action, reward, next_state):
        self.memory.append([state, action, reward, next_state])


    # build the replay buffer 저장한 것을 버퍼에서 꺼내오는.? 학습단계?
    def replay(self,allowed_actions):
        
        global mse_loss
        mse=[]
        batch_size = 32
        #list_of_next_allowed_machines = []
        if len(self.memory) < batch_size: #buffer에 저장된 memory가 buffer의 총 batch_size보다 작다면 return
            return 
        samples = random.sample(self.memory, batch_size) #메모리에서 배치사이즈만큼 랜덤으로 선택
        for sample in samples:
            state, action, reward, new_state = sample # sample 데이터 하나를 꺼내서
            #print (state)
            state = np.array(state).reshape(1,len(state)) 
            #print("state",state)
            new_state = np.array(new_state).reshape(1,len(new_state))
            #print("new_state",new_state) #????????????
            target = self.target_model.predict(state) #기존 state로 target을 예측
            
            action_id = allowed_actions.index(tuple(action)) #63개의 allowed action 중에서 state에 대한 action의 index를 추출

            target[0][action_id]=reward #63개의 action을 예측한 target에서 state에 대한 실제 action에 reward를 할당
                
            
            
            next_pred = self.target_model.predict(new_state)[0] #new state에 대한 target 예측
            
            new_action=self.maxQ_action(next_pred,new_state,allowed_actions) # new state에 대한 action예측
            
            Q_future=next_pred[allowed_actions.index(new_action)] #next state에 대한 action의 Q값은?
                
            target[0][action_id] = reward + Q_future * self.learning_rate # target의 action_id번째 위치에 다음 Q값이 들어감. 맞춰야 하는 값!!!
      
            
            history=self.model.fit(state, target, epochs= 1, verbose= 0) #1에폭으로 학습
            
            mse.append(history.history['loss'][0])
            
        mse_loss.append(min(mse))
        
        #print("Mean_square_error:"min(mse_loss))
        


    # update our target network 
    def train_target(self): #학습에서 가중치를 업데이트
        print("update weights")
        weights = self.model.get_weights() 
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)#loss함수?
        self.target_model.set_weights(target_weights)



    # save our model 
    def save_model(self, fn):
        self.model.save(fn)


# Initialize the policy and the target network

        
def episode(env,DQN,Tsc,Tfu,allowed_actions): #pacing rate가 각 flow에게 할당
    global action
    global state
    global request
    global flow_success
    fs=request['filesize']
    cnt=1
    c=0
    dqn_agent = DQN()
    while True:#에피소드 시작
        print ("-------------")
        print("Episode start",cnt)
        print ("")
        first_action=1
        
        while ((request['filesize'][0]!=0)or(request['filesize'][1]!=0)or(request['filesize'][2]!=0)):#모두 전송이 완료될 때 까지
            c+=1
#             print("")
#             print ("Scheduling interval : ",c)
#             print("")
            #scheduling interval의 초기에 action을 update하고, 끝날 때 reward를 할당한다.
            #state는 각 source의 min_rate로서, 남은 filesize/남은 deadlin까지의 기한 을 뜻한다.
            #deadline이 0인 경우(이미 초과하거나 완료된 경우) 0으로 나누기 때문에 에러 발생. 즉 deadline이 0이면 이전 state를 할당
            #또한, float형을 int형으로 형변환해버림으로서 반올림이 아니라 버림을 함 -> 따라서 math패키지의 ceil()을 활용해 올림연산
            try: 
                state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources']))]
            except:#deadline이 0인 경우 error
                for s in range(len(sources)):
                    if (request['deadline'][s] != 0):
                        state[s]= math.ceil(request['filesize'][s]/request['deadline'][s])
                    else:
                        state[s]=0

            #print("현재 sc에서의 state",state)

    #         for i in range(len(sources)):#전송 완료된 source가 있다면 action은 0으로 할당
    #             if (request['filesize'][i]==0):
    #                 action[i]=0

            if (first_action==1): #New state-action policy를 EDF방식으로 (가장 deadline이 시급한)     
                action=[0,0,0]
                index=np.argmin(request['deadline']) #deadline 최소인 source의 index
                action[index]=10 #bottleneck capacity
                
            else: 
                action=dqn_agent.choose_action(state,allowed_actions) #action이 None으로 return되는 현상?(수정: min_rate이상의 allowed_action이 없었기 때문)
            #print("현재 sc에서의 action",action)    


            for i in range(Tsc):#하나의 scheduling interval 내의 flow update interval
                #print("flow update interval :",env.now)
                #print("전송중!영차영차")

                for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경(각source에 대한 반복)
    #                 print ("filesize of source",s)
    #                 print (request['filesize'][s])
    #                 print ("deadline of source",s)
    #                 print (request['deadline'][s])
                   
                    request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
                    request['deadline'][s]=max([request['deadline'][s]-1,0])
                    if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                        #print ("s{}의 전송이 완료됨".format(s))
                        if (request['deadline'][s]!=0): #기간 안에 전송되면?
                            request['value'][s]=1 #value를 1로 변경
                            flow_success.append(1)
                            print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                        else: #기간안에 전송된게 아니라면(value초기값은 None)
                            request['value'][s]=0
                            flow_success.append(0)
                            print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
                yield env.timeout(Tfu)
            
            
            print ("action", action)
            print ("")
            print ("source 0 전송할 file:",request['filesize'][0])
            print ("")
            print ("source 1 전송할 file:",request['filesize'][1])
            print ("")
            print ("source 2 전송할 file:",request['filesize'][2])

            try: 
                next_state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources']))]
            except:#deadline이 0인 경우 error
                for s in range(len(sources)):
                    if (request['deadline'][s] != 0):
                        next_state[s]= math.ceil(request['filesize'][s]/request['deadline'][s])
                    else:
                        next_state[s]=0
                        
            #print(next_state)
                        
            # build your reward function 리워드 방정식 입력
            reward = sum((np.max(request['deadline'])-request['deadline'])*action)
            cur_state = state 
            action = action
            new_state = next_state 
            #new_action = next_allowed_actions
            reward = reward
            #done = done
            
            if (first_action==0): 
                dqn_agent.remember(cur_state, action, reward, new_state) #새로운 state로 설정해주고 기존state저장
                dqn_agent.replay(allowed_actions)#학습
                dqn_agent.train_target()
            
            first_action=0 

        for i in range(len(sources)):
            if (request['value'][i]==1):
                print ("source {} 전송완료".format(i))
            else:
                print ("source {} deadine 충족하지 못함".format(i))
        
        print("모두 전송 완료")

        filesize=[random.randrange(10,50) for source in range(len(sources))]
        deadline=[int(filesize[source]/2.6) for source in range(len(sources))] #sum_rmin이 8이 나올 정도
        request = {
            'sources' : sources,
            'destinations' : destinations, 
            'filesize' : filesize, #단위는 Gbps
            'deadline' : deadline,
            'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
        }#다음 episode를 위해 request 생성

        state=[0,0,0]#초기 state
        action=[0,0,0]#초기 action ->고칠것
        print ("episode end")
        print("-----------------")
        cnt+=1
    
    dqn_agent.save_model("policy_network.h5")
    
    


#한 번의 에피소드 프로세스
#N개의 source에서 동시에 request를 발생시키면 시작됨
#끝나는 기점은 모든 flow가 전송이 완료되는 시점
#랜덤으로 filesize를 생성, deadline도 랜덤으로 생성
env = simpy.Environment()

mse_loss=[] #episode의 진행에 따른 mse_loss의 변화율 graph
flow_success=[] #episode의 진행에 따른 flow_success rate


state=[0,0,0]#초기 state
action=[0,0,0]#초기 action ->고칠것
Tsc=5 #scheduling interval을 구성하는 flow update interval의 수
Tfu=1 #flow update interval의 시간단위(1초로 가정)
sources = ['s0', 's1', 's2']
destinations = ['d0', 'd1', 'd2']
filesize=[random.randrange(10,50) for source in range(len(sources))]
deadline=[int(filesize[source]/2.6) for source in range(len(sources))] #sum_rmin이 8이 나올 정도
request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : filesize, #단위는 Gbps
    'deadline' : deadline,
    'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

A=list(range(11))
B=list(range(11))
C=list(range(11)) #합이 10이 되는 0~10까지의 수 조합
allowed_actions=[]
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 63개의 조합

# print ("source 0의 min_rate:",math.ceil(request['filesize'][0]/request['deadline'][0]))
# print("")
# print ("source 1의 min_rate:",math.ceil(request['filesize'][1]/request['deadline'][1]))
# print("")
# print ("source 0의 min_rate:",math.ceil(request['filesize'][2]/request['deadline'][2]))


env.process(episode(env,DQN,Tsc,Tfu,allowed_actions))
env.run(until=100000)#10만 초 동안 가동
print("종료")

#print (mse_loss)
#print (flow_success)


-------------
Episode start 1

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 47

source 1 전송할 file: 30

source 2 전송할 file: 0
random action
action (6, 4, 0)

source 0 전송할 file: 17

source 1 전송할 file: 10

source 2 전송할 file: 0
update weights
random action
s1의 전송이 deadline을 지나 완료됨
action (3, 5, 2)

source 0 전송할 file: 2

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
random action
s0의 전송이 deadline 안에 완료됨
action (1, 0, 9)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 2

s1의 전송이 deadline 안에 완료됨
action [0, 10, 0]

source 0 전송할 file: 46

source 1 전송할 file: 0

source 2 전송할 file: 49
random action
action (5, 5, 0)

source 0 전송할 file: 21

source 1 전송할 file: 0

source 2 전송할 file: 49
update weights
random action
action (3, 7, 0)

source 0 전송할 file: 6

source 1 전송할 file: 0

source 2 전송할 file: 49
update weights
random actio

update weights
state [0 1 5]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
s1의 전송이 deadline 안에 완료됨
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 27
update weights
state [0 0 9]
q: [0]
s2의 전송이 deadline을 지나 완료됨
action (0, 0, 10)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 15

s0의 전송이 deadline 안에 완료됨
action [10, 0, 0]

source 0 전송할 file: 0

source 1 전송할 file: 22

source 2 전송할 file: 21
state [0 8 7]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
s1의 전송이 deadline을 지나 완료됨
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 11
update weights
random action
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 11
update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
sour

update weights
state [2 0 6]
q: [0, 11, 21, 30, 38, 45, 51, 56, 60, 63, 65]
s0의 전송이 deadline 안에 완료됨
action (7, 0, 3)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 8
update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 28

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 21

source 1 전송할 file: 44

source 2 전송할 file: 0
state [7 4 0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline을 지나 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 44

source 2 전송할 file: 0
update weights
state [0 8 0]
q: [10]
s1의 전송이 deadline 안에 완료됨
action (0, 10, 0)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 deadine 충족하지 못함
source 1 전송완료
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------

update weights
state [0 1 6]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
s1의 전송이 deadline 안에 완료됨
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 20
update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 42

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 44

source 1 전송할 file: 30

source 2 전송할 file: 0
state [4 5 0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline 안에 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 30

source 2 전송할 file: 0
update weights
state [ 0 30  0]
q: [10]
s1의 전송이 deadline을 지나 완료됨
action (0, 10, 0)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
episode end
-----------------
------------

update weights
source 0 전송완료
source 1 전송완료
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 55

s1의 전송이 deadline 안에 완료됨
action [0, 10, 0]

source 0 전송할 file: 22

source 1 전송할 file: 0

source 2 전송할 file: 43
state [8 0 4]
q: [0, 11, 21, 30, 38, 45, 51, 56, 60, 63, 65]
s0의 전송이 deadline을 지나 완료됨
action (7, 0, 3)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 28
update weights
state [0 0 5]
q: [0]
s2의 전송이 deadline 안에 완료됨
action (0, 0, 10)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 deadine 충족하지 못함
source 1 전송완료
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 56

s0의 전송이 deadline 안에 완료됨
action [10, 0, 0]

source 0 전송할 file: 0

source 1 전송할 file: 41

source 2 전송할 file: 29
state [0 5 5]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 1

source 2 전송할 file: 19
update weights
state [ 0  1 19]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 

...

update weights
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 124

s1의 전송이 deadline 안에 완료됨
action [0, 10, 0]

source 0 전송할 file: 22

source 1 전송할 file: 0

source 2 전송할 file: 24
state [8 0 6]
q: [0, 11, 21, 30, 38, 45, 51, 56, 60, 63, 65]
s0의 전송이 deadline을 지나 완료됨
action (7, 0, 3)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 9
update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 deadine 충족하지 못함
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 125

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 32

source 1 전송할 file: 43

source 2 전송할 file: 0
state [5 4 0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline 안에 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 43

source 2 전송할 file: 0
update weigh

update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 138

s1의 전송이 deadline 안에 완료됨
action [0, 10, 0]

source 0 전송할 file: 30

source 1 전송할 file: 0

source 2 전송할 file: 29
state [5 0 5]
q: [0, 11, 21, 30, 38, 45, 51, 56, 60, 63, 65]
s0의 전송이 deadline 안에 완료됨
action (7, 0, 3)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 14
update weights
state [ 0  0 14]
q: [0]
s2의 전송이 deadline을 지나 완료됨
action (0, 0, 10)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 139

s0의 전송이 deadline 안에 완료됨
action [10, 0, 0]

source 0 전송할 file: 0

source 1 전송할 file: 46

source 2 전송할 file: 36
state [0 4 5]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9

update weights
state [0 1 6]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
s1의 전송이 deadline 안에 완료됨
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 24
update weights
state [ 0  0 24]
q: [0]
s2의 전송이 deadline을 지나 완료됨
action (0, 0, 10)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 151

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 20

source 1 전송할 file: 17

source 2 전송할 file: 0
state [10 17  0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline을 지나 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 17

source 2 전송할 file: 0
update weights
state [0 0 0]
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 2

source 2 전송할 file: 0
update weights
state [0 0 0]
s1의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
upd

update weights
state [0 0 0]
s1의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 164

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 46

source 1 전송할 file: 39

source 2 전송할 file: 0
state [4 4 0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline 안에 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 39

source 2 전송할 file: 0
update weights
state [0 8 0]
q: [10]
s1의 전송이 deadline 안에 완료됨
action (0, 10, 0)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 165

s1의 전송이 deadline 안에 완료됨
action [0, 10, 0]

source 0 전송할 file: 38

source 1 전송할 file: 0

source 2 전송할 file: 43
state [5 0 4]
q: [0, 11, 21, 30, 38, 45, 51, 56, 60, 63, 6

update weights
state [0 1 9]
q: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
s1의 전송이 deadline 안에 완료됨
action (0, 8, 2)

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 16
update weights
state [0 0 0]
s2의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 178

s2의 전송이 deadline 안에 완료됨
action [0, 0, 10]

source 0 전송할 file: 18

source 1 전송할 file: 18

source 2 전송할 file: 0
state [18 18  0]
q: [10, 20, 29, 37, 44, 50, 55, 59, 62, 64, 65]
s0의 전송이 deadline을 지나 완료됨
action (10, 0, 0)

source 0 전송할 file: 0

source 1 전송할 file: 18

source 2 전송할 file: 0
update weights
state [0 0 0]
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 3

source 2 전송할 file: 0
update weights
state [0 0 0]
s1의 전송이 deadline을 지나 완료됨
action [3, 3, 4]

source 0 전송할 file: 0

source 1 전송할 file: 0

source 2 전송할 file: 0
update weights

In [9]:
flow_success

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [10]:
rate=[]
flow_success=np.array(flow_success)
for i in range(int(len(flow_success)/10)):#10개당평균
    print(flow_success[10*i:10*(i+1)])
    rate.append(np.mean(flow_success[10*i:10*(i+1)]))
    print(np.mean(flow_success[10*i:10*(i+1)]))


[1 0 0 1 0 0 1 0 1 1]
0.5
[0 0 1 1 0 1 1 0 1 0]
0.5
[0 1 1 1 1 0 0 1 1 0]
0.6
[1 0 0 1 1 0 1 1 0 1]
0.6
[1 0 1 0 0 1 1 0 1 1]
0.6


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# x = mse_loss

# plt.plot(x)
# plt.show()

#flow_success_rate=
x2 = rate

plt.plot(x2)
plt.show()

In [74]:
len(mse_loss)

1968

1.할 것 : flow update interval을 줄이기 (0.001초?) 어느정도가 적당할 지 교수님꼐여줘보기

2.reward의 문제점: drem(deadline까지 남아있는 시간)이 가장 짧은 source에 많은 pacing rate를 할당할 수록 높은 reward가 주어지는데, file size를 고려하지 않았다는 문제점이 있다. deadline이 아니라 min rate를 기준으로 해야할듯

3.state=0인 경우 filesize=0이란뜻이므로 action을 0으로 할당하는 코드를 짜야함. 문제는 두개의 filesize가 0인경우 어떻게 하냐는것...!!!!! -> state가 0인 index에 따라 action도 0인 allowed_action의 index를 찾고, 그 중 가장 큰 maxQ값을 갖는 action 선택

4.10만초 동안 가동

In [28]:
A=list(range(11))
B=list(range(11))
C=list(range(11)) #합이 10이 되는 0~10까지의 수 조합
allowed_actions=[]
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 66개의 조합
        
allowed_actions.index((0,10,0))
#print (allowed_actions)

10

In [12]:
state=np.array([3,3,3])
np.array(state).reshape(1,state.shape[0])

array([[3, 3, 3]])

In [82]:
#deeplearning없이 시뮬레이션
env = simpy.Environment()

request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : [random.randrange(50,100) for source in sources], #단위는 Gbps
    'deadline' : [random.randrange(5,30) for source in sources], 
    'value' : [None for source in sources], #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

print ("source 0의 filesize:",request['filesize'][0])
print ("source 1의 filesize:",request['filesize'][1])
print ("source 2의 filesize:",request['filesize'][2])

action=[3,3,4]

Tfu=1

def flowUpdate_interval(env,action,Tfu): #Tsc에서의 action(pacint rate)가 n번의 flow update interval만큼 유지된다.
    #print("flow update interval :",env.now)
    global request
    fs=request['filesize']
    #src=request['sources']
    while True:
        print("flow update interval :",env.now)
        if ((fs[0]!=0)or(fs[1]!=0)or(fs[2]!=0)): #하나라도 0이상으로 남아있으면
            
            print("전송중!영차영차")
            
            for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경
                request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
                request['deadline'][s]=max([request['deadline'][s]-1,0])
                if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                    #print ("s{}의 전송이 완료됨".format(s))
                    if (request['deadline'][s]!=0): #기간 안에 전송되면?
                        request['value'][s]=1 #value를 1로 변경
                        print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                    else: #기간안에 전송된게 아니라면(value초기값은 None)
                        request['value'][s]=0
                        print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
            
            yield env.timeout(Tfu) 
        elif ((fs[0]==0)and(fs[1]==0)and(fs[2]==0)):
            print ("종료:",env.now)
            return #모든 flow의 전송이 완료되면 종료
        
        
proc=env.process(flowUpdate_interval(env,action,Tfu))
env.run(until=proc)

source 0의 filesize: 87
source 1의 filesize: 56
source 2의 filesize: 78
flow update interval : 0
전송중!영차영차
flow update interval : 1
전송중!영차영차
flow update interval : 2
전송중!영차영차
flow update interval : 3
전송중!영차영차
flow update interval : 4
전송중!영차영차
flow update interval : 5
전송중!영차영차
flow update interval : 6
전송중!영차영차
flow update interval : 7
전송중!영차영차
flow update interval : 8
전송중!영차영차
flow update interval : 9
전송중!영차영차
flow update interval : 10
전송중!영차영차
flow update interval : 11
전송중!영차영차
flow update interval : 12
전송중!영차영차
flow update interval : 13
전송중!영차영차
flow update interval : 14
전송중!영차영차
flow update interval : 15
전송중!영차영차
flow update interval : 16
전송중!영차영차
flow update interval : 17
전송중!영차영차
flow update interval : 18
전송중!영차영차
s1의 전송이 deadline 안에 완료됨
flow update interval : 19
전송중!영차영차
s2의 전송이 deadline 안에 완료됨
flow update interval : 20
전송중!영차영차
flow update interval : 21
전송중!영차영차
flow update interval : 22
전송중!영차영차
flow update interval : 23
전송중!영차영차
flow update interval : 24
전송중!영차영차
flow update interv